In [ ]:
!pip install unsloth -q

In [ ]:
!pip install tensorrt_llm -U --pre --extra-index-url https://pypi.nvidia.com
!pip install transformers -U

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import re
import torch

from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextIteratorStreamer, AutoTokenizer
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             classification_report, confusion_matrix)
from tqdm.auto import tqdm
from tensorrt_llm.runtime import ModelRunner

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# **Download Dataset**

In [ ]:
dataset = load_dataset("dair-ai/emotion")

In [ ]:
train_dataset = dataset["train"].to_pandas()
test_dataset = dataset["test"].to_pandas()
val_dataset = dataset["validation"].to_pandas()

# **Load Model**

In [ ]:
base_model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

# **Transformed Dataset**

In [ ]:
emotion_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

emotion_list = "['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']"

In [ ]:
def create_question(text):
    prompt = f"""
    <task>
    Perform a deep emotional reasoning analysis on the provided sentence.
    </task>

    <sentence>
    {text}
    </sentence>
    """
    return prompt


def generate_cot(text, emotion):
    templates = {
        'sadness': f"""
    1. Analyze input: "{text}"
    2. Identify keywords: Look for terms related to loss, grief, disappointment, or hopelessness.
    3. Assess tone: The tone is heavy, somber, or melancholic.
    4. Reasoning: The speaker expresses a sense of unhappiness or emotional pain regarding a situation.
    5. Conclusion: The primary emotion is sadness.
    """,

        'joy': f"""
    1. Analyze input: "{text}"
    2. Identify keywords: Look for terms related to happiness, success, excitement, or gratitude.
    3. Assess tone: The tone is uplifting, enthusiastic, or positive.
    4. Reasoning: The speaker conveys a state of well-being, pleasure, or satisfaction.
    5. Conclusion: The primary emotion is joy.
    """,

        'love': f"""
    1. Analyze input: "{text}"
    2. Identify keywords: Look for terms related to affection, devotion, intimacy, or caring.
    3. Assess tone: The tone is warm, tender, or passionate.
    4. Reasoning: The speaker expresses deep attachment or strong positive feelings towards someone or something.
    5. Conclusion: The primary emotion is love.
    """,

        'anger': f"""
    1. Analyze input: "{text}"
    2. Identify keywords: Look for terms related to frustration, irritation, hostility, or rage.
    3. Assess tone: The tone is aggressive, harsh, or defensive.
    4. Reasoning: The speaker shows strong displeasure or antagonism towards a trigger or person.
    5. Conclusion: The primary emotion is anger.
    """,

        'fear': f"""
    1. Analyze input: "{text}"
    2. Identify keywords: Look for terms related to anxiety, danger, panic, or uncertainty.
    3. Assess tone: The tone is nervous, tense, or urgent.
    4. Reasoning: The speaker anticipates a threat or feels unsafe about a future or current event.
    5. Conclusion: The primary emotion is fear.
    """,

        'surprise': f"""
    1. Analyze input: "{text}"
    2. Identify keywords: Look for terms related to shock, disbelief, suddenness, or unexpected events.
    3. Assess tone: The tone is startled, amazed, or confused (neutral to positive/negative).
    4. Reasoning: The speaker is reacting to something unforeseen or contrary to expectations.
    5. Conclusion: The primary emotion is surprise.
    """
    }

    return templates.get(emotion, "")

In [ ]:
def transform_data_train(data: pd.DataFrame) -> pd.DataFrame:
    transformed_data = pd.DataFrame(columns=['Question', 'Complex_CoT', 'Response'])

    transformed_data["Response"] = data["label"].apply(lambda x: emotion_map[x])
    transformed_data["Question"] = data["text"].apply(create_question)
    transformed_data["Complex_CoT"] = data.apply(
        lambda row: generate_cot(row["text"], emotion_map[row["label"]]),
        axis=1
    )

    return transformed_data

def mapping_response(data: pd.DataFrame) -> pd.DataFrame:
    data['label'] = data['label'].apply(lambda x: emotion_map[x])

    return data

In [ ]:
transformed_train = transform_data_train(train_dataset).reset_index(drop=True)
transformed_test = mapping_response(test_dataset)
transformed_val = mapping_response(val_dataset)

# **Train Model**

## **Prepare Dataset for Training**

In [ ]:
train_prompt_style = r"""Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and emotionally insightful response.

### Instruction:
You are an expert in emotional intelligence and sentiment analysis.
Analyze the given sentence and identify the primary emotion expressed by the speaker.
Follow the Cognitive Appraisal Theory framework to evaluate the speaker's goals and agency.
Choose only ONE emotion from the provided list: sadness, joy, love, anger, fear, surprise.

Please follow these steps:
1. Analyze the keywords, tone, and context within <think> tags.
2. Determine the most dominant emotion based on your analysis.
3. Output the final emotion explicitly within <Emotions> tags.

### Question:
{}

### Response:
<think>
{}
</think>

Final Answer:
<Emotions>{}</Emotions>"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token

In [ ]:
def formatting_prompts_func(examples):
    questions = examples["Question"]
    cots = examples["Complex_CoT"]
    responses = examples["Response"]
    texts = []

    for question, cot, response in zip(questions, cots, responses):
        text = train_prompt_style.format(question, cot, response) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

train_dataset = Dataset.from_pandas(transformed_train)
train_dataset_modeling = train_dataset.map(formatting_prompts_func, batched=True)

print("\n📄 Sample formatted text:")
print(train_dataset_modeling["text"][0][:])

## **Setup LoRA PEFT**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2026.2.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## **Setup Training Arguments**

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset_modeling,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch = 8
        warmup_steps=10,
        num_train_epochs=1,  # Atau pakai max_steps
        max_steps=100,  # Alternatif: fixed steps
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        report_to="none",
    ),
)

In [ ]:
trainer_stats = trainer.train()

# **Create Prediction**

In [ ]:
inference_prompt_style = r"""Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and emotionally insightful response.

### Instruction:
You are an expert in emotional intelligence and sentiment analysis.
Analyze the given sentence and identify the primary emotion expressed by the speaker.
Follow the Cognitive Appraisal Theory framework to evaluate the speaker's goals and agency.
Choose only ONE emotion from the provided list: sadness, joy, love, anger, fear, surprise.

Please follow these steps:
1. Analyze the keywords, tone, and context within <think> tags.
2. Determine the most dominant emotion based on your analysis.
3. Output the final emotion explicitly within <Emotions> tags.

### Question:
{}

### Response:
<think>
"""

In [ ]:
emotion_list = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

def extract_emotion(response_text, emotion_list):
    try:
        emotion_pattern = r'<Emotions>\s*([^<]+?)\s*</Emotions>'
        emotion_match = re.search(emotion_pattern, response_text, re.IGNORECASE | re.DOTALL)

        if emotion_match:
            emotion_content = emotion_match.group(1).strip()

            emotion_content = emotion_content.strip('.,;:!? \n\r\t').lower()

            emotion_list_lower = [e.lower() for e in emotion_list]
            if emotion_content in emotion_list_lower:
                return emotion_content

            words = emotion_content.split()
            for word in words:
                clean_word = word.strip('.,;:!?').lower()
                if clean_word in emotion_list_lower:
                    return clean_word

        text_without_think = re.sub(r'<think>.*?</think>', '', response_text, flags=re.DOTALL | re.IGNORECASE)

        pattern = r'\b(' + '|'.join(emotion_list) + r')\b'
        matches = re.findall(pattern, text_without_think, re.IGNORECASE)
        if matches:
            return matches[-1].lower()

        return "unknown"

    except Exception as e:
        print(f"Error: {e}")
        return "unknown"

In [ ]:
question = create_question("im feeling quite sad and sorry for myself but ill snap out of it soon")

FastLanguageModel.for_inference(model)

input_text = inference_prompt_style.format(question)
inputs = tokenizer([input_text], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
    stop_strings=["<｜end▁of▁sentence｜>", "</s>"],
    tokenizer=tokenizer
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)
print(f"Output dari extract emotion : {extract_emotion(generated_text, emotion_list)}")

# **Evaluate Model**

In [ ]:
def predict_batch(texts, model, tokenizer, prompt_style, emotion_list, batch_size=8):
    predictions = []

    tokenizer.padding_side = "left"

    FastLanguageModel.for_inference(model)

    for i in tqdm(range(0, len(texts), batch_size), desc="Predicting"):
        batch_texts = texts[i:i+batch_size]

        try:
            questions = [create_question(text) for text in batch_texts]

            prompts = [inference_prompt_style.format(q) for q in questions]

            # Tokenize
            inputs = tokenizer(
                prompts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=2048
            ).to("cuda")

            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=1024,
                use_cache=True,
                stop_strings=["<｜end▁of▁sentence｜>", "</s>"],
                tokenizer=tokenizer
            )

            generated_ids = outputs[:, inputs.input_ids.shape[1]:]
            responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

            for response_text in responses:

                emotion = extract_emotion(response_text, emotion_list)
                predictions.append(emotion)

        except Exception as e:
            print(f"\nError processing batch {i//batch_size + 1}: {str(e)}")
            predictions.extend(["unknown"] * len(batch_texts))
            continue

    tokenizer.padding_side = "right"

    if len(predictions) != len(texts):
        print(f"Warning: Got {len(predictions)} predictions for {len(texts)} texts")

    return predictions

In [ ]:
texts = transformed_val['text'].tolist()
true_labels = transformed_val['label'].tolist()

predictions = predict_batch(texts, model, tokenizer, inference_prompt_style, emotion_list, batch_size=32)

transformed_val['predicted'] = predictions

In [ ]:
def calculate_metrics(true_labels, predictions, emotion_list):
    # Filter unknown predictions
    valid_indices = [i for i, pred in enumerate(predictions) if pred != "unknown"]

    if len(valid_indices) == 0:
        print("No valid predictions")
        return None

    filtered_true = [true_labels[i] for i in valid_indices]
    filtered_pred = [predictions[i] for i in valid_indices]

    # Accuracy
    accuracy = accuracy_score(filtered_true, filtered_pred)

    # Precision, Recall, F1
    precision, recall, f1, support = precision_recall_fscore_support(
        filtered_true, filtered_pred, labels=emotion_list, average=None, zero_division=0
    )

    # Macro averages
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
        filtered_true, filtered_pred, average='macro', zero_division=0
    )

    # Confusion Matrix
    cm = confusion_matrix(filtered_true, filtered_pred, labels=emotion_list)

    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'support': support,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'confusion_matrix': cm,
        'valid_count': len(valid_indices),
        'total_count': len(predictions),
        'unknown_count': len(predictions) - len(valid_indices)
    }

    return results

In [ ]:
results = calculate_metrics(true_labels, predictions, emotion_list)

if results:
    print("="*60)
    print("EVALUATION RESULTS")
    print("="*60)
    print(f"\nTotal samples: {results['total_count']}")
    print(f"Valid predictions: {results['valid_count']}")
    print(f"Unknown predictions: {results['unknown_count']}")
    print(f"\nOverall Accuracy: {results['accuracy']:.4f}")
    print(f"\nMacro Precision: {results['macro_precision']:.4f}")
    print(f"Macro Recall: {results['macro_recall']:.4f}")
    print(f"Macro F1-Score: {results['macro_f1']:.4f}")

    print("\n" + "="*60)
    print("PER-CLASS METRICS")
    print("="*60)
    print(f"{'Emotion':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<12}")
    print("-"*60)

    for i, emotion in enumerate(emotion_list):
        print(f"{emotion:<12} {results['precision'][i]:<12.4f} {results['recall'][i]:<12.4f} {results['f1'][i]:<12.4f} {results['support'][i]:<12}")

In [ ]:
def plot_confusion_matrix(cm, emotion_list):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_list, yticklabels=emotion_list)
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.show()

plot_confusion_matrix(results['confusion_matrix'], emotion_list)

# **Convert Model to TensorRT**

In [ ]:
model.save_pretrained_merged(
    "model_merged_hf",
    tokenizer,
    save_method = "merged_16bit",
)

In [ ]:
!mkdir -p trt_checkpoint

!python -m tensorrt_llm.commands.convert_checkpoint \
    --model_dir ./model_merged_hf \
    --output_dir ./trt_checkpoint \
    --dtype float16 \
    --tp_size 1 \
    --workers 1

In [ ]:
!mkdir -p trt_engine

!trtllm-build \
    --checkpoint_dir ./trt_checkpoint \
    --output_dir ./trt_engine \
    --gemm_plugin float16 \
    --gpt_attention_plugin float16 \
    --max_batch_size 8 \
    --max_input_len 2048 \
    --max_output_len 1024

In [ ]:
runner = ModelRunner.from_dir("./trt_engine")


input_text = inference_prompt_style.format("I am feeling sad today.")

# Generate
outputs = runner.generate(
    [input_text],
    max_new_tokens=500,
    end_id=151643,
    pad_id=151643
)

tokenizer = AutoTokenizer.from_pretrained("./model_merged_hf")

output_ids = outputs[0][0].tolist() # Batch 0, Beam 0
text = tokenizer.decode(output_ids, skip_special_tokens=True)

print(text)